## 4. 데이터 정제
모델이 데이터를 원활하게 학습하기 위해서는 잘 정제되어 있는 데이터셋이 필요하다. 결측치 대체, 범주형 데이터의 수치화등의 데이터 정제등을 필수적으로 거쳐야 모델 학습시 유리하다.

### (1) 타겟변수와 예측변수 분리
정제대상은 예측변수 데이터들이므로 우선 타겟변수와 예측변수를 분리한다.

In [61]:
housing = strat_train_set.drop(columns = 'median_house_value', axis = 1)
housing_label = strat_train_set['median_house_value'].copy()

### (2) 결측치 대치를 위한 Imputer객체 생성
해당 컬럼에 결측치가 존재한다면 이를 중앙값과 같은 다른 수치로 대체할 수 있다. 이를 위해서 sklearn의 Imputer객체를 생성한다. Imputer객체의 fit메서드를 이용하면 Imputer.statistics_에 해당 통계수치값들이 저장되며 transform메서드를 이용해서 결측치를 해당 통계수치값들로 대체할 수 있다. 단 Transform메서드 사용시 반환되는 값은 데이터프레임 형식이 아니므로 추후 pandas의 DataFrame메서드를 이용해 형식을 데이터프레임으로 바꿔야함에 주의하자

In [62]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = "median")

In [63]:
housing_num = housing.drop(columns = 'ocean_proximity')

In [64]:
imputer.fit(housing_num)
imputer.statistics_

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [65]:
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns = housing_num.columns, index = housing_num.index)

### (3) 문자형식으로 구성되어 있는 범주형 데이터 변환
모델학습이 원활하게 이루어지기 위해서는 모든 데이터가 수치형데이터여야 한다. 문자형식으로 이루어진 데이터의 경우 sklearn.preprocessing의 OrdinalEncoder이나 OneHotCoding을 사용한다. 단 OrdinalEncoder의 경우 모델이 변환된 수치들간의 잘못된 규칙을 학습할 수 있으니 주의하자.

In [66]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

,ocean_proximity
12655,INLAND
15502,NEAR OCEAN
2908,INLAND
14053,NEAR OCEAN
20496,<1H OCEAN
1481,NEAR BAY
18125,<1H OCEAN
5830,<1H OCEAN
17989,<1H OCEAN
4861,<1H OCEAN


In [67]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[1.],
       [4.],
       [1.],
       [4.],
       [0.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [68]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 16512 stored elements and shape (16512, 5)>

In [69]:
housing_cat_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [70]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]